 <div style="background-color: #99CD4E; text-align:center; vertical-align: middle; padding:40px 0;"> 
  <h1 style="color: white;"> *Text Similarity* </h1>.
 </div>

<div style="background-color: #99CD4E; padding:5px 0;"> 
  <h2 style="color: white;"> *Text Similarity Use Cases* </h1>
 </div>
 
Find and eliminate duplicate documents to focus on key issues, and save time. For example, 

- eliminate duplicate alerts in network management.  
- remove duplicate news items for analsysts.


Finding similar items for recommendations

<div style="background-color: #99CD4E; padding:5px 0;"> 
  <h2 style="color: white;"> *Similarity Metric* </h1>
 </div>

See [Mining of Massive Datasets](http://infolab.stanford.edu/~ullman/mmds/ch3.pdf) Chapter 3.5 for text similarity measures


Given two vectors _*X*_ and _*y*_ with _*p*_ dimensions the distance between them is defined as follows:

_**Eulidean Distance**_

$$\sqrt{(x_1 - y_1)^2 + (x_2 - y_2)^2  + \ldots + (x_p - y_p)^2  }$$

_**Cosine Similarity**_

$$\frac{\sum\limits_{k = 1}^{p} xy }{ \sqrt{\sum\limits_{k = 1}^{p} x^2} \sqrt{\sum\limits_{k = 1}^{p} y^2} }  $$

cosine similarity gives distance 1 when items are close, and -1 when items are apart.

_**Jaccard Distance**_

$$1 - \frac{| x \cap y |}{| x \cup y |}$$


# Get book text

NLTK includes a small selection of texts from the Project Gutenberg electronic text archive which we are going to use in this exercse


In [1]:
from nltk.corpus import gutenberg
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [2]:
books = ['austen-emma.txt', 'austen-persuasion.txt','austen-sense.txt',
         'carroll-alice.txt','shakespeare-caesar.txt','shakespeare-hamlet.txt', 'shakespeare-macbeth.txt']

# Word frequencies

In [3]:
import nltk

stopwords = nltk.corpus.stopwords.words('english')

for book in books:
    print(book)
    words = gutenberg.words(book)
    fd = nltk.FreqDist(w.lower() for w in words if w.lower() not in stopwords if w.isalpha())
    print(fd.most_common(10))  

austen-emma.txt
[('mr', 1153), ('emma', 865), ('could', 837), ('would', 820), ('mrs', 699), ('miss', 599), ('must', 567), ('harriet', 506), ('much', 486), ('said', 484)]
austen-persuasion.txt
[('anne', 497), ('could', 451), ('would', 355), ('captain', 303), ('mrs', 291), ('elliot', 289), ('mr', 256), ('one', 238), ('must', 228), ('wentworth', 218)]
austen-sense.txt
[('elinor', 685), ('could', 578), ('marianne', 566), ('mrs', 530), ('would', 515), ('said', 397), ('every', 377), ('one', 331), ('much', 290), ('must', 283)]
carroll-alice.txt
[('said', 462), ('alice', 398), ('little', 128), ('one', 104), ('know', 88), ('like', 85), ('went', 83), ('would', 83), ('could', 77), ('queen', 75)]
shakespeare-caesar.txt
[('caesar', 190), ('brutus', 161), ('bru', 153), ('haue', 148), ('shall', 125), ('thou', 115), ('cassi', 107), ('cassius', 85), ('antony', 75), ('come', 74)]
shakespeare-hamlet.txt
[('ham', 337), ('lord', 211), ('haue', 178), ('king', 172), ('thou', 107), ('shall', 107), ('come', 10

# Convert text into tf-idf vector

In [4]:
print(type(gutenberg.sents('austen-emma.txt')))

<class 'nltk.corpus.reader.util.StreamBackedCorpusView'>


In [5]:
total_text = []
for book in books:
    total_text.append(gutenberg.raw(book))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_total_text = tfidf_vectorizer.fit_transform(total_text)

In [7]:
print(type(tfidf_total_text))

<class 'scipy.sparse.csr.csr_matrix'>


In [8]:
import numpy as np
print(np.shape(tfidf_total_text))

(7, 15812)


# Compute cosine similarity matrix 

cosine_similarity(...) create a square matrix in which the number of rows and columns is equal to the number of documents. dist[i, j] gives the cosine similarity between i and j titles.

cosine similarity gives distance 1 when items are close, and -1 when items are aprt. In the clustering algorithms higher the distance means that items are similar. Hence, we subtract the computed distance from 1.

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
dist_matrix = 1 - cosine_similarity(tfidf_total_text)   

# Find closest match for each book

In [11]:
dist_matrix

array([[ -1.33226763e-15,   6.01504152e-01,   6.42827294e-01,
          8.42037481e-01,   9.27488486e-01,   9.21508140e-01,
          9.21809453e-01],
       [  6.01504152e-01,   0.00000000e+00,   6.69388161e-01,
          8.52648683e-01,   9.29014624e-01,   9.20352323e-01,
          9.13800868e-01],
       [  6.42827294e-01,   6.69388161e-01,  -4.44089210e-16,
          8.46323063e-01,   9.32399745e-01,   9.23216094e-01,
          9.22817889e-01],
       [  8.42037481e-01,   8.52648683e-01,   8.46323063e-01,
          0.00000000e+00,   9.51557089e-01,   9.30205766e-01,
          9.38386775e-01],
       [  9.27488486e-01,   9.29014624e-01,   9.32399745e-01,
          9.51557089e-01,   3.33066907e-16,   6.73658270e-01,
          6.30364873e-01],
       [  9.21508140e-01,   9.20352323e-01,   9.23216094e-01,
          9.30205766e-01,   6.73658270e-01,  -2.22044605e-15,
          5.70279666e-01],
       [  9.21809453e-01,   9.13800868e-01,   9.22817889e-01,
          9.38386775e-01,   6.30

In [28]:
## Hack to ensure that diganol with self distance is not minimum value
n = len(dist_matrix)
dist_matrix[range(n), range(n)] = 1
dist_matrix

array([[ 1.        ,  0.60150415,  0.64282729,  0.84203748,  0.92748849,
         0.92150814,  0.92180945],
       [ 0.60150415,  1.        ,  0.66938816,  0.85264868,  0.92901462,
         0.92035232,  0.91380087],
       [ 0.64282729,  0.66938816,  1.        ,  0.84632306,  0.93239975,
         0.92321609,  0.92281789],
       [ 0.84203748,  0.85264868,  0.84632306,  1.        ,  0.95155709,
         0.93020577,  0.93838677],
       [ 0.92748849,  0.92901462,  0.93239975,  0.95155709,  1.        ,
         0.67365827,  0.63036487],
       [ 0.92150814,  0.92035232,  0.92321609,  0.93020577,  0.67365827,
         1.        ,  0.57027967],
       [ 0.92180945,  0.91380087,  0.92281789,  0.93838677,  0.63036487,
         0.57027967,  1.        ]])

In [29]:
import pandas as pd
dist_matrix_data_frame = pd.DataFrame(dist_matrix, columns=books, index=books)

In [30]:
dist_matrix_data_frame

,austen-emma.txt,austen-persuasion.txt,austen-sense.txt,carroll-alice.txt,shakespeare-caesar.txt,shakespeare-hamlet.txt,shakespeare-macbeth.txt
austen-emma.txt,1.000000,0.601504,0.642827,0.842037,0.927488,0.921508,0.921809
austen-persuasion.txt,0.601504,1.000000,0.669388,0.852649,0.929015,0.920352,0.913801
austen-sense.txt,0.642827,0.669388,1.000000,0.846323,0.932400,0.923216,0.922818
carroll-alice.txt,0.842037,0.852649,0.846323,1.000000,0.951557,0.930206,0.938387
shakespeare-caesar.txt,0.927488,0.929015,0.932400,0.951557,1.000000,0.673658,0.630365
shakespeare-hamlet.txt,0.921508,0.920352,0.923216,0.930206,0.673658,1.000000,0.570280
shakespeare-macbeth.txt,0.921809,0.913801,0.922818,0.938387,0.630365,0.570280,1.000000


In [31]:
dist_matrix_data_frame['Minimum'] = dist_matrix_data_frame.apply(lambda x: x.argmin(), axis=1)

In [32]:
dist_matrix_data_frame['Minimum']

austen-emma.txt              austen-persuasion.txt
austen-persuasion.txt              austen-emma.txt
austen-sense.txt                   austen-emma.txt
carroll-alice.txt                  austen-emma.txt
shakespeare-caesar.txt     shakespeare-macbeth.txt
shakespeare-hamlet.txt     shakespeare-macbeth.txt
shakespeare-macbeth.txt     shakespeare-hamlet.txt
Name: Minimum, dtype: object

# Exercise

- Try to repeat this exercise with all the books, and see which books are closest to them

- Repeat this exercise with Eucledian distance and evalaute the results especially when compared to cosine distance

 <div style="background-color: #99CD4E; text-align:center; vertical-align: middle; padding:40px 0;"> 
  <h1 style="color: white;"> *The End* </h1>.
 </div>